In [1]:
import os
import numpy as np
import torch
import random
from torchvision import transforms
from normalize import Normalize, MapToRange
from torch.utils.data import Dataset, DataLoader

from torch import nn
# from torch_nn import *

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
torch.set_num_threads(1)

print(torch.cuda.is_available())

True


**Preprocessing the dataset**

In [2]:
class TrajectoryDataset(Dataset):
    
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        i, j = self.indices[index]        
        X = torch.tensor([
            self.dataset['dx'][i, j],
            self.dataset['dy'][i, j],
            self.dataset['dz'][i, j],
            self.dataset['vx'][i, j],
            self.dataset['vy'][i, j],
            self.dataset['vz'][i, j],
            self.dataset['phi'][i, j],
            self.dataset['theta'][i, j],
            self.dataset['psi'][i, j],
            self.dataset['p'][i, j],
            self.dataset['q'][i, j],
            self.dataset['r'][i, j],
            self.dataset['omega'][i, j, 0],
            self.dataset['omega'][i, j, 1],
            self.dataset['omega'][i, j, 2],
            self.dataset['omega'][i, j, 3],
            self.dataset['Mx_ext'][i],
            self.dataset['My_ext'][i],
            self.dataset['Mz_ext'][i]
        ], dtype=torch.float32)
        
        U = torch.tensor([
            self.dataset['u'][i, j, 0],
            self.dataset['u'][i, j, 1],
            self.dataset['u'][i, j, 2],
            self.dataset['u'][i, j, 3]
        ], dtype=torch.float32)
        
        return X, U
    
# trajectories containing 199 points
dataset_path = 'datasets/hover_dataset.npz'

dataset = dict()
print('loading dataset...')
with np.load(dataset_path) as full_dataset:
    # total number of trajectories
    num = len(full_dataset['dx'])
    print(num, 'trajectories')
    dataset = {key: full_dataset[key] for key in [
        't', 'dx', 'dy', 'dz', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'p', 'q', 'r','omega', 'u', 'omega_min','omega_max', 'k_omega', 'Mx_ext', 'My_ext', 'Mz_ext'
    ]}

# train/test split
batchsize_train = 256
batchsize_val = 4096
train_trajectories = range(int(0.0008*num))
test_trajectories = list(set(range(int(0.001*num))) - set(train_trajectories))

train_indices = [(i, j) for i in train_trajectories for j in range(199)]
train_set = TrajectoryDataset(dataset, train_indices)
train_loader = DataLoader(train_set, batch_size=batchsize_train, shuffle=True, num_workers=1)

test_indices = [(i, j) for i in test_trajectories for j in range(199)]
test_set = TrajectoryDataset(dataset, test_indices)
test_loader = DataLoader(test_set, batch_size=batchsize_val, shuffle=True, num_workers=1)

print('ready')

print('Amount of testing trajectories: ',len(test_trajectories),f'(Batchsize: {batchsize_val})')
print('Amount of training trajectories: ',len(train_trajectories),f'(Batchsize: {batchsize_train})')

loading dataset...
100000 trajectories
ready
Amount of testing trajectories:  200 (Batchsize: 4096)
Amount of training trajectories:  800 (Batchsize: 256)


In [3]:
print(len(test_trajectories))
print(len(train_trajectories))

print(dataset['omega_min'])
print(dataset['omega_max'])

200
800
5000.0
10000.0


**Calculating mean and standard deviation for normalization**

In [4]:
from tqdm import tqdm
X_mean = torch.zeros(19)
X_std = torch.zeros(19)

N=10000

for i, data in tqdm(enumerate(test_set)):
    X = data[0]
    X_mean += X
    if i>=N:
        break
X_mean = X_mean/N

print('mean:')
print(X_mean)
    
for i, data in tqdm(enumerate(test_set)):
    X = data[0]
    X_std += (X-X_mean)**2
    if i>=N:
        break

X_std = torch.sqrt(X_std/N)
print('std:')
print(X_std)

10000it [00:00, 48252.15it/s]


mean:
tensor([ 3.3776e-01, -6.2278e-02, -5.3854e-01,  2.0858e-01,  6.7812e-04,
        -1.0187e-01, -2.5429e-02, -4.3297e-02, -9.9357e-02, -3.1766e-02,
        -5.0724e-02,  1.2835e-01,  7.6618e+03,  7.6040e+03,  7.7439e+03,
         7.7066e+03, -5.8896e-04,  4.6322e-04,  5.7101e-04])


10000it [00:00, 26539.44it/s]

std:
tensor([1.4792e+00, 1.5088e+00, 1.0129e+00, 1.4844e+00, 1.5017e+00, 6.1984e-01,
        3.0464e-01, 3.0117e-01, 1.0474e+00, 1.6662e+00, 1.5671e+00, 1.1332e+00,
        9.7876e+02, 1.0069e+03, 9.8156e+02, 1.0100e+03, 2.2238e-02, 2.4357e-02,
        5.8174e-03])


**Defining the neural network**

In [5]:
model = nn.Sequential(
    Normalize(mean=X_mean, std=X_std),
    nn.Linear(19, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 4),
    nn.Sigmoid()
)
model

c:\Users\tavar\Desktop\Thesis\Code\quad_SL\normalize.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean, dtype=torch.float32)
c:\Users\tavar\Desktop\Thesis\Code\quad_SL\normalize.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std, dtype=torch.float32)


Sequential(
  (0): Normalize()
  (1): Linear(in_features=19, out_features=120, bias=True)
  (2): ReLU()
  (3): Linear(in_features=120, out_features=120, bias=True)
  (4): ReLU()
  (5): Linear(in_features=120, out_features=120, bias=True)
  (6): ReLU()
  (7): Linear(in_features=120, out_features=4, bias=True)
  (8): Sigmoid()
)

**Testing**

In [6]:
x1 = torch.randn(19)
print(model(x1))
# print([param.shape for param in model.parameters()])

tensor([0.3154, 0.4505, 0.7365, 0.4318], grad_fn=<SigmoidBackward0>)


**Define a Loss function and optimizer**

In [7]:
criterion = torch.nn.MSELoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=1, verbose=True, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)


**Training loop**

In [8]:
from tqdm import tqdm
import time
import copy

loss_list = []
loss_val_list = []
best_loss = 0.1
first = True
start_time = time.time()

# loop over the dataset multiple times
num_epochs = 100

nn_model_name = f"{dataset_path[9:-4]}_{batchsize_train}_{batchsize_val}_{learning_rate}_{num_epochs}"

for epoch in range(num_epochs):
    
    if first:
        time_remaining = '-'
    else:
        time_estimate = epoch_time*(num_epochs-epoch+1)
        if time_estimate > 60:
            if time_estimate > 3600:
                time_remaining = str(round(time_estimate/3600,2))+' h'
            else:
                time_remaining = str(round(time_estimate/60,2))+' min'
        else:
            time_remaining = str(round(time_estimate,0))+' s'
        
    first = False
    print(f"Epoch {epoch+1}/{num_epochs}, Current learning rate: {optimizer.state_dict()['param_groups'][0]['lr']}, Time remaining: {time_remaining}")

    start_time_epoch = time.time()
    
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    
    for i, (data, targets) in loop:
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        
        # Loss
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Update progressbar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
        loss_list.append(loss.item())

    # Validate
    with torch.no_grad():
        # Get a random batch from the test dataset
        data_val, targets_val = next(iter(test_loader))

        # Forward pass
        outputs_val = model(data_val)

        # Loss
        loss_val = criterion(outputs_val, targets_val)

        if loss_val < best_loss:
            # Save best model
            best_model = copy.deepcopy(model)
            
            # Backup
            torch.save(model, 'neural_networks/tmp_benchmark.pt')
            
            best_loss = loss_val
            print("Best model updated!")

        # Scheduler (reduce learning rate if loss stagnates)
        scheduler.step(loss_val)
        
        loss_val_list.append(loss_val.item())

    print(f'loss = {loss:.8f}, loss validation = {loss_val:.8f} '+r' (control error: +/-'+str(round(100*np.sqrt(float(loss_val)),2))+'%)\n')

    epoch_time = (time.time() - start_time_epoch)

    loop.close()
    
# Compute excecution time
execution_time = (time.time() - start_time)    
print(f"Total training time: {round(execution_time,2)}s")

# Save best model and copy for maptorange network
torch.save(best_model, f'neural_networks/{nn_model_name}.pt')
best_model_for_maptorange = torch.load('neural_networks/tmp_benchmark.pt')
print(best_model_for_maptorange)

Epoch 1/100, Current learning rate: 0.0001, Time remaining: -


**Testing performance**

In [ ]:
from tqdm import tqdm

loader = test_loader
# loop over the test dataset
loop = tqdm(enumerate(loader), total=len(loader), leave=False)
running_loss = 0

for i, (data, targets) in loop:
    outputs = model(data)
    loss = criterion(outputs, targets)
    
    running_loss += loss.item()
    
    # update progressbar
    loop.set_postfix(loss=loss.item())

loop.close()
print('average loss =', running_loss/len(loader))

**Saving model**

In [ ]:
torch.save(model, 'neural_networks/HOVER_TO_HOVER_NOMINAL.pt')

In [ ]:
model = torch.load('neural_networks/HOVER_TO_HOVER_NOMINAL.pt')

In [ ]:
model_processed_output =nn.Sequential(
    *model,
    MapToRange(dataset['omega_min'], dataset['omega_max'])
)

print(model_processed_output)
torch.save(model_processed_output, 'neural_networks/HOVER_TO_HOVER_NOMINAL_.pt')